In [111]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer 
from sklearn.svm import OneClassSVM
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from  sklearn.linear_model import RidgeClassifier 
from  sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split

In [112]:
def get_dataDF(
   
    ):
    
    features = { 
        'age':  'numerical',
        'w_edu':'categorical',      
        'h_edu':'categorical',     
        'num_child': 'numerical',
        'religion': 'categorical',           
        'w_working': 'categorical',     
        'h_occupation': 'categorical',       
        'life_index': 'categorical',
        'media': 'categorical'
    }                                                

    print(list(features.keys()))
    cat_features = [ _ for _, t in features.items()  if t == 'categorical' ]
    numeric_features = [ _ for _, t in features.items()  if t =='numerical' ]
    

    df = pd.read_csv('./cmc.data', 
                     header=None, 
                     names= list(features.keys()) + ['label'],
                     index_col=None
        )
    
    labels = df.loc[:,'label'].values
    le = LabelEncoder()
    le.fit(labels)
    enc_labels = le.transform(labels)
    df.loc[:,'label'] = enc_labels
    label_classes = le.classes_

    return df, numeric_features, cat_features, label_classes

In [113]:
df, numeric_features, cat_features, label_classes = get_dataDF()

['age', 'w_edu', 'h_edu', 'num_child', 'religion', 'w_working', 'h_occupation', 'life_index', 'media']


In [142]:
df.loc[df['label']==1].life_index.describe()

count    333.000000
mean       3.468468
std        0.770149
min        1.000000
25%        3.000000
50%        4.000000
75%        4.000000
max        4.000000
Name: life_index, dtype: float64

In [115]:
df.head(10)

,age,w_edu,h_edu,num_child,religion,w_working,h_occupation,life_index,media,label
0,24,2,3,3,1,1,2,3,0,0
1,45,1,3,10,1,1,3,4,0,0
2,43,2,3,7,1,1,3,4,0,0
3,42,3,2,9,1,1,3,3,0,0
4,36,3,3,8,1,1,3,2,0,0
5,19,4,4,0,1,1,3,3,0,0
6,38,2,3,6,1,1,3,2,0,0
7,21,3,3,1,1,0,3,2,0,0
8,27,2,3,3,1,1,3,4,0,0
9,45,1,1,8,1,1,2,2,1,0


,w_edu,life_index,count
0,1,1,29
1,1,2,38
2,1,3,45
3,1,4,40
4,2,1,55
5,2,2,72
6,2,3,100
7,2,4,107
8,3,1,37
9,3,2,81


In [116]:
categorical_names = { c:list(sorted(set(df[c].values))) for  c in cat_features}

In [117]:
categorical_catLabels = {k: list(range(len(v))) for k,v in categorical_names.items()}

In [118]:
for cat, cat_lables in categorical_catLabels.items():
    df[cat].astype(CategoricalDtype(cat_lables))

In [119]:
df_01enc = df.copy()

In [120]:
for cat in cat_features:
    df_01enc = pd.concat(
        [df_01enc,
         pd.get_dummies(df_01enc[cat], 
                        prefix=cat
                       )],axis=1
    ).drop([cat],axis=1) 

In [121]:
df_01enc.head(10)

,age,num_child,label,w_edu_1,w_edu_2,w_edu_3,w_edu_4,h_edu_1,h_edu_2,h_edu_3,...,h_occupation_1,h_occupation_2,h_occupation_3,h_occupation_4,life_index_1,life_index_2,life_index_3,life_index_4,media_0,media_1
0,24,3,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,45,10,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
2,43,7,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
3,42,9,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
4,36,8,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
5,19,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
6,38,6,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
7,21,1,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
8,27,3,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
9,45,8,0,1,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [47]:
numFeature_Scalers = {}
for num_f in numeric_features:
    obj= MinMaxScaler()
    obj.fit(df_01enc.loc[:,num_f].values.reshape([-1,1]))
    df_01enc.loc[:,num_f]= obj.transform( df_01enc.loc[:,num_f].values.reshape([-1,1])).reshape(-1)

In [62]:
tmp  = df_01enc.loc[df_01enc['label']!=2]
tmp = tmp.sample(frac=1.0)
Y = tmp['label']
del tmp['label']
X = tmp.values

In [129]:
model = RidgeClassifier(solver='lsqr',fit_intercept=False)

In [ ]:
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.25)
model.fit(train_X, train_Y)
pred = model.predict(test_X),test_Y
accuracy_score(test_Y.values, pred[0])

In [134]:
test_Y.values, pred[0]

(array([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]),
 array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1

In [124]:
model.coef_.shape

(1, 24)

In [169]:
tmp1  =  df.loc[df['label']==0]
tmp1.head(10)

,age,w_edu,h_edu,num_child,religion,w_working,h_occupation,life_index,media,label
0,24,2,3,3,1,1,2,3,0,0
1,45,1,3,10,1,1,3,4,0,0
2,43,2,3,7,1,1,3,4,0,0
3,42,3,2,9,1,1,3,3,0,0
4,36,3,3,8,1,1,3,2,0,0
5,19,4,4,0,1,1,3,3,0,0
6,38,2,3,6,1,1,3,2,0,0
7,21,3,3,1,1,0,3,2,0,0
8,27,2,3,3,1,1,3,4,0,0
9,45,1,1,8,1,1,2,2,1,0


In [170]:

tmp1.groupby(['h_edu','life_index']).size().reset_index(name='count')

,h_edu,life_index,count
0,1,1,9
1,1,2,10
2,1,3,7
3,1,4,5
4,2,1,27
5,2,2,27
6,2,3,23
7,2,4,22
8,3,1,19
9,3,2,42
